api call

In [21]:
from openai import OpenAI
from tqdm import tqdm
import httpx
import pandas as pd
import openai
import backoff

api_key = "sk-ShmyeH9VjAnRuT1S55A71a9fC69640948d20F73bA634C3A5"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=5)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content.strip('\n')


instruction_hoang = '''You will be provided with some log messages delimited by backticks. You should check if the given log messages share the same template.If so, abstract variables with ‘{placeholders}’ and return the template without additional explatation, otherwise return the template list'''


instruction0 = '''Gving some log raws, the AI assistant can parse them to one or more templates: {"template": ['template_1']}.The "templates" field (list) denotes the templates that the log contents use. You should abstract variables with ‘{placeholders}’ and only return the dictionary'''

instruction2 = '''Giving some log tempaltes from some log contents which may use the same template, the AI assistant should merge the similar templates into a single template'''

instruction_noindex = '''You will be provided with some log messages. You should check if the giving log messages share the same template. If so, abstract variables with ‘{placeholders}’ and return the template without additional explatation, otherwise return the templates'''

instruction_100 = '''do the following log contents share the same template? Return the template or templates without additional explatation'''

instruction_1_13 = '''You will be provided with some log messages. You should check whether the giving log messages share the same template. If so, abstract variables with ‘{placeholders}’ and return the template without additional explatation'''

instruction_1_13_2 = '''Given some log templates, you should choice the most corret one. If you think none of them is correct, you can return the correct tempalte you think'''

prompt = '''Block broadcast_9 stored as values in memory (estimated size 8.8 KB, free 14.0 KB)
Block broadcast_8 stored as values in memory (estimated size 281.6 KB, free 317.0 KB)
Block broadcast_10 stored as values in memory (estimated size 9.6 KB, free 339.2 KB)
Block broadcast_11 stored as values in memory (estimated size 9.2 KB, free 353.9 KB)
Block broadcast_12 stored as values in memory (estimated size 9.8 KB, free 369.3 KB)
Block broadcast_13 stored as values in memory (estimated size 10.1 KB, free 386.5 KB)
Block broadcast_14 stored as values in memory (estimated size 9.6 KB, free 401.9 KB)
Block broadcast_15 stored as values in memory (estimated size 9.2 KB, free 416.6 KB)
Block broadcast_16 stored as values in memory (estimated size 9.8 KB, free 432.0 KB)
Block broadcast_17 stored as values in memory (estimated size 10.1 KB, free 448.6 KB)
Block broadcast_18 stored as values in memory (estimated size 9.6 KB, free 464.0 KB)
Block broadcast_19 stored as values in memory (estimated size 9.2 KB, free 478.6 KB)
Block broadcast_20 stored as values in memory (estimated size 9.8 KB, free 411.6 KB)
Block broadcast_21 stored as values in memory (estimated size 10.1 KB, free 403.2 KB)
Block broadcast_22 stored as values in memory (estimated size 9.6 KB, free 418.6 KB)
Block broadcast_23 stored as values in memory (estimated size 9.2 KB, free 387.4 KB)
Block broadcast_24 stored as values in memory (estimated size 9.8 KB, free 402.8 KB)'''

prompt2 = '''<*>:<*> open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS
<*>: <*> open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS
<*>:<*> open through proxy <*>:<*> HTTPS
<*>:<*> open through proxy proxy.cse.cuhk.edu.hk:5070 <*>
<*>:<*> open through proxy proxy.<*>:<*> HTTPS
<*>:443 open through proxy <*>:5070 HTTPS
<*>:<*> open through proxy <*>:<*> <*>
<*>:open through proxy <*>:HTTPS'''

messages = [{"role": "system", "content": instruction_1_13_2}]
messages.append({"role": "user", "content": prompt2})
print(get_responce(messages))

Based on the log templates provided, the most correct one is:

<*>:<*> open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS


Evaluate_gpt

In [1]:
from evaluator import evaluate
import pandas as pd
dataset = 'Linux'
file = f'outputs/enhanced_gpt/1shot/{dataset}.csv'
evaluate(file, dataset)

Precision: 0.9738, Recall: 0.5896, F1_measure: 0.7345, Group Accuracy: 0.2385, Message-Level Accuracy: 0.6090, Edit Distance: 1.4800


(0.2385, 0.609, 1.48, 2.9768439663509403)

验证按' '分词有多少词，以及含数字的词占多少

In [15]:
import re
import pandas as pd
datasets = ['BGL' ,'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac','Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
a = 0
b = 0
list = []
for dataset in datasets:
    df = pd.read_csv(f"dataset\{dataset}\{dataset}_2k.log_templates_corrected.csv")
    list = df['EventTemplate'].tolist()    
    total_words = 0
    words_with_numbers = 0

    for s in list:
        words = s.split()
        total_words += len(words)
        words_with_numbers += len([word for word in words if re.search(r'\d', word)])
    a+=total_words
    b+=words_with_numbers

print(f"{b} / {a}")

109 / 9841


In [2]:
import json
# load demonstrations
with open('demonstrations.json', 'r') as f:
    demonstrations = json.load(f)
messages = []
print(demonstrations['Spark'][0])

{'role': 'user', 'content': 'Log: Block broadcast_0 stored as values in memory (estimated size 384.0 B, free 317.5 KB)'}


聚类结果的自定义处理

In [19]:
import re

def replace_words(s):
    words = s.split(' ')
    for i in range(len(words)):
        if words[i].count('<*>') >= 2:
            if words[i].endswith(':'):
                words[i] = '<*>:'
            elif words[i].endswith(','):
                words[i] = '<*>,'
            elif words[i].endswith('.'):
                words[i] = '<*>.'
            else:
                words[i] = '<*>'
    # zookeeper
    # output_str = input_str.replace(' /<*>', ' <*>')
    return ' '.join(words)

# 测试


inputs = '''<*>:<*> open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS
<*>: <*> open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS
<*>:<*> open through proxy <*>:<*> HTTPS
<*>:<*> open through proxy proxy.cse.cuhk.edu.hk:5070 <*>
<*>:<*> open through proxy proxy.<*>:<*> HTTPS
<*>:443 open through proxy <*>:5070 HTTPS
<*>:<*> open through proxy <*>:<*> <*>
<*>:open through proxy <*>:HTTPS'''

lines = inputs.split('\n')
for line in lines:
    print(replace_words(line))

<*> open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS
<*>: <*> open through proxy proxy.cse.cuhk.edu.hk:5070 HTTPS
<*> open through proxy <*> HTTPS
<*> open through proxy proxy.cse.cuhk.edu.hk:5070 <*>
<*> open through proxy <*> HTTPS
<*>:443 open through proxy <*>:5070 HTTPS
<*> open through proxy <*> <*>
<*>:open through proxy <*>:HTTPS
